In [4]:
from scratch import query_high_low_quantile
import polars as pl
from polars import col as c
import polars.selectors as cs

## Purpose

Identify trends in pharmacy margin using NADAC as an estimated cost basis.

## Key questions
- Are there systematic differences in reimbursement between PBM-affiliated and non‑affiliated pharmacies?
- Which providers lie in the tails of the margin distribution (1st and 99th percentiles)?

## Data sources
- WV OIG PBM NADAC Reporting
- CMS NADAC Reporting
- Claims dataset with fields including: `total`, `nadac`, `affiliate`, `ndc`, `drug`, `date`

## Definitions
- **Pharmacy margin** = `total - nadac` (positive => paid more than NADAC; negative => paid below NADAC).
- **affiliate** = whether the pharmacy is affiliated with the PBM (as provided in the dataset).

## Methods (summary)
1. Compute `margin = total - nadac` at the claim level and create aggregated summaries.
2. Group results by `affiliate` and compare central tendency (median), spread (IQR), and tails (1st/99th percentiles).
3. Visualize distributions (boxplots/violins), time trends, and list top outliers by margin and volume for manual review.

## Outputs
- Summary table by `affiliate`: count, median margin, 1st/99th percentiles.
- Visualizations: distribution plots and time series of margins.
- CSV with top N outlier providers for follow-up.

## Caveats
- NADAC is an estimate of acquisition cost and may not reflect discounts, rebates, or special pricing agreements.

In [ ]:
(
query_high_low_quantile(0.01, 0.99)
.collect(engine='streaming')
.style
.tab_header('Margin Over NADAC', subtitle='Top 1% and Bottom 1% of Claims')
.fmt_currency(cs.matches('(?i)low|high|net$'), accounting=True)
.fmt_percent(cs.matches('(?i)pct'), decimals=0)
.cols_width({
    'low': '150px',
    'high': '150px',
    'net': '150px',
    'net_pct_change': '150px'
})
)

C:\Users\mwine\AppData\Local\Temp\ipykernel_3380\4199347043.py:2: UserWarning: Sortedness of columns cannot be checked when 'by' groups provided
  query_high_low_quantile(0.01, 0.99)


GT(_tbl_data=shape: (2, 5)
┌───────────┬────────┬────────┬────────┬────────────────┐
│ affiliate ┆ low    ┆ high   ┆ net    ┆ net_pct_change │
│ ---       ┆ ---    ┆ ---    ┆ ---    ┆ ---            │
│ bool      ┆ f64    ┆ f64    ┆ f64    ┆ f64            │
╞═══════════╪════════╪════════╪════════╪════════════════╡
│ false     ┆ -27.69 ┆ 146.69 ┆ 119.0  ┆ null           │
│ true      ┆ -119.5 ┆ 445.43 ┆ 325.93 ┆ 1.738908       │
└───────────┴────────┴────────┴────────┴────────────────┘, _body=<great_tables._gt_data.Body object at 0x0000015C08A07EE0>, _boxhead=Boxhead([ColInfo(var='affiliate', type=<ColInfoTypeEnum.default: 1>, column_label='affiliate', column_align='center', column_width=None), ColInfo(var='low', type=<ColInfoTypeEnum.default: 1>, column_label='low', column_align='right', column_width='150px'), ColInfo(var='high', type=<ColInfoTypeEnum.default: 1>, column_label='high', column_align='right', column_width='150px'), ColInfo(var='net', type=<ColInfoTypeEnum.default: 1>, column_label='net', column_align='right', column_width='150px'), ColInfo(var='net_pct_change', type=<ColInfoTypeEnum.default: 1>, column_label='net_pct_change', column_align='right', column_width='150px')]), _stub=<great_tables._gt_data.Stub object at 0x0000015C089EBBE0>, _spanners=Spanners([]), _heading=Heading(title='Margin Over NADAC', subtitle='Top 1% and Bottom 1% of Claims', preheader=None), _stubhead=None, _source_notes=[], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x0000015C03F7D930>, _formats=[<great_tables._gt_data.FormatInfo object at 0x0000015C089EE9E0>, <great_tables._gt_data.FormatInfo object at 0x0000015C089EDA20>], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_additional_css=OptionsInfo(scss=False, category='table', type='values', value=[]), table_font_names=OptionsInfo(scss=False, category='table', type='values', value=['-apple-system', 'BlinkMacSystemFont', 'Segoe UI', 'Roboto', 'Oxygen', 'Ubuntu', 'Cantarell', 'Helvetica Neue', 'Fira Sans', 'Droid Sans', 'Arial', 'sans-serif']), table_font_size=OptionsInfo(scss=True, category='table', type='px', value='16px'), table_font_weight=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_style=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_color=OptionsInfo(scss=True, category='table', type='value', value='#333333'), table_font_color_light=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_border_top_include=OptionsInfo(scss=False, category='table', type='boolean', value=True), table_border_top_style=OptionsInfo(scss=True, category='table', type='value', value='solid'), table_border_top_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_top_color=OptionsInfo(scss=True, category='table', type='value', value='#A8A8A8'), table_border_right_style=OptionsInfo(scss=True, category='table', type='value', value='none'), table_border_right_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_right_color=OptionsInfo(scss=True, category='table', type='value', value='#D3D3D3'), table_border_bottom_include=OptionsInfo(scss=False, category='table', type='boolean', value=True), table_border_bottom_style=OptionsInfo(scss=True, category='table', type='value', value='solid'), table_border_bottom_width=OptionsInfo(scss=True, category='table',